In [1]:
# Data
import pandas as pd
import numpy as np

# Warnings
import warnings
warnings.simplefilter("ignore", UserWarning)

# Models
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# These models are voting models based off the above models
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingRegressor

# Data prep
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Model evaluations
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold,StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance

# Charts
import pandas_bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
pd.set_option('plotting.backend', 'pandas_bokeh')
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
output_notebook()

Loading BokehJS ...

In [2]:
# Models

svc = SVC(kernel='rbf', gamma=0.1, C=10) # 5% increase with these hyperparamters
KNC = KNeighborsClassifier(weights='distance', p=2, n_neighbors=10, metric='euclidean', leaf_size=40) # 2.7% increase with these hp
ADBC = AdaBoostClassifier(n_estimators=155, learning_rate=0.8) # 2% increase with these hp
RFC = RandomForestClassifier(n_estimators=1000, min_samples_split=5) # 1% better with these hyperparameters

GBC = GradientBoostingClassifier(n_estimators=500, learning_rate=0.15) # 2% better
HGBC = HistGradientBoostingClassifier(min_samples_leaf=25, max_leaf_nodes=80, max_iter=100, max_depth=None, learning_rate=0.1, l2_regularization=1.5) # 2% better
XGB = XGBClassifier(n_estimators=150, learning_rate=0.1) # 1.7% better with hp
QDA = QuadraticDiscriminantAnalysis() # Same with default hp

# Imputer
imputer = SimpleImputer()
MMScaler = MinMaxScaler()

In [22]:
# What year are you using?
year = '2018-19'

# Use last years data?
use_last_year_data = True

# How many months of the current year to train on?
months_to_train = 0

# Keep the year before discard years after
if use_last_year_data:
    year_before = int(year[:4]) - 1
    year_before_that = str(year_before - 1)
else:
    year_before = int(year[:4])
    year_before_that = str(year_before -1)

# get data from NBA_Combined
data = pd.read_excel('./content/NBA_COMBINED.xlsx', sheet_name='Games', parse_dates=['Date'])

# Get data from Final_odds_2020-2021.csv
odds = pd.read_csv(f'./Web Scraping/Final_Odds_{str(year)}.csv', parse_dates=['Date'])

float_cols = [col for col in odds.columns if col not in ['Date', 'Home', 'Visitor']]
for col in float_cols:
    odds[col] = odds[col].astype('float')


# Keep necessary dates

if year == '2019-20':
    end_month = '10'
    start_month = '10'
elif year == '2020-21':
    start_month = '12'
else:
    start_month = '10'
    end_month = '07'

# year_before = int(year[:4]) - 1
year_after = pd.to_datetime(str(year_before + 2) + '-' + end_month, format='%Y-%m')
year_before = pd.to_datetime(str(year_before) + '-' + start_month, format='%Y-%m')

filt = (data['Date'] > year_before) & (data['Date'] < year_after) #### THIS WOULD INCLUDE THE ENDS OF THE PREVIOUS SEASON AND THE BEGGINING OF NEXT SEASON. HAVE TO CUT OFF AT MONTH AS WELL
data = data.loc[filt]

rows_with_results = len(data) - len(data[data['Vis PTS'].isna()])
data = data.iloc[:rows_with_results-1,:]

# Remove unnecesary columns. All data before 2017. Change columns names to Prev Year.
data.drop(['PTS22', 'PTS3', 'Attend.'], axis=1, inplace=True)

print('Removing year columns we do not need')

str_year_before = str(year_before)[:4]

year_cols = ['2016-17 Vis Rank', '2016-17 Home Rank', '2017-18 Vis Rank', '2017-18 Home Rank', '2018-19 Vis Rank', '2018-19 Home Rank', '2019-20 Vis Rank', '2019-20 Home Rank']

cols_to_remove = [x for x in year_cols if str_year_before not in x]
cols_to_remove = [x for x in cols_to_remove if year_before_that not in x]

data.drop(cols_to_remove, inplace=True, axis=1)

# Convert points column to int
data = data.astype({'Vis PTS': int, 'Home PTS': int})

# Merge DataFrames
odds.drop('Date', axis=1, inplace=True) # Odds portal fucks its dates up
data = data.merge(odds)

# Create y / ['Home Win'] column
data['Home Win'] = data['Home PTS'] > data['Vis PTS']


# ADD TIMES AND DATES

print('Adding Times and Dates')

# Get Day, Month and Year from date column
dates = pd.DataFrame()
dates['Year'] = data['Date'].dt.strftime('%Y')
dates['Month'] = data['Date'].dt.strftime('%m')
dates['Day'] = data['Date'].dt.strftime('%d')

# Add dates
data = pd.concat([data, dates], axis=1)

# Get start time
start_time = data['Start (ET)'].str[:-1]
start_time = start_time.str.replace(':', '.')
start_time = start_time.astype(float)
start_time.columns = ['Start Time']

# Add start time
data = pd.concat([data, start_time], axis=1)


# ADD HOME/VIS LAST WIN STREAK

print('Adding Who Won Last')

data["HomeLastWin"] = False
data["VisitorLastWin"] = False

from collections import defaultdict
won_last = defaultdict(int) # Create dictionary won last

for index, row in data.iterrows(): # for each row
  home_team = row['Home'] # Take the home team in the row
  visitor_team = row['Visitor'] # Take the vis team in each row
  row['HomeLastWin'] = won_last[home_team] # If HomeLastWin is true set that team to won in the won_last dict
  row['VisitorLastWin'] = won_last[visitor_team] # If VisitorLastWin is true set that team to won in won_last dict
  data.loc[index] = row # Set the index for the next row?
  # Set current win
  won_last[home_team] = row['Home Win'] # If home won set that in the won_last dict
  won_last[visitor_team] = not row['Home Win'] # if home did not win set that in the won_last dict
    
# ADD WINSTREAKS

print('Adding Win Streaks')

data['HomeWinStreak'] = 0
data['VisitorWinStreak'] = 0

win_streak = defaultdict(int) #  Create a dictionary for teams winning streaks

for index, row in data.iterrows():
  home_team = row['Home'] # Home team = home team for that row
  visitor_team = row['Visitor'] # Vis team = vis team for that row
  row['HomeWinStreak'] = win_streak[home_team] # HomeWinStreak for that row is looked up in the dictionary win_streak
  row['VisitorWinStreak'] = win_streak[visitor_team] # Set VisitorWinStreak in the row to dict value for that team
  data.loc[index] = row # Set row to next row
  # Set current win streak number
  if row['Home Win']:
    win_streak[home_team] += 1
    win_streak[visitor_team] = 0
  else:
    win_streak[home_team] = 0
    win_streak[visitor_team] += 1
    
# WHICH TEAM WON LAST MATCH

print('Adding which team won last')

last_match_winner = defaultdict(int)

def home_team_won_last(row):
  # Variables equal the team names
  home_team = row['Home']
  visitor_team = row['Visitor']

  teams = tuple(sorted([home_team, visitor_team])) # Tuple of the home and visitor team to search for
  result = 1 if last_match_winner[teams] == row['Home'] else 0 # Look in last_match_winner dict for if these teams have played before
  winner = row['Home'] if  row['Home Win'] else row['Visitor'] # Winner variable is home team if the homewin column says it is

  last_match_winner[teams] = winner # Feed the winner into the last_match_winner dict

  return result

data['HomeTeamWonLast'] = data.apply(home_team_won_last, axis=1) # Apply the function on each row (axis=1)


# Add in power rankings

print('Adding power rankings.')

str_year_before = str(year_before)[:4]

power_rankings_df = pd.read_csv('./Web Scraping/Power_rankings.csv')

home_power_rankings_df = power_rankings_df[['Name', f'{str_year_before} WPower', f'{str_year_before} LPower', f'{str_year_before} GFPower', f'{str_year_before} GAPower', f'{str_year_before} MPower']]
vis_power_rankings_df = power_rankings_df[['Name', f'{str_year_before} WPower', f'{str_year_before} LPower', f'{str_year_before} GFPower', f'{str_year_before} GAPower', f'{str_year_before} MPower']]

home_power_rankings_df.columns = ['Home', f'{str_year_before} H WPower', f'{str_year_before} H LPower', f'{str_year_before} H GFPower', f'{str_year_before} H GAPower', f'{str_year_before} H MPower']
vis_power_rankings_df.columns = ['Visitor', f'{str_year_before} V WPower', f'{str_year_before} V LPower', f'{str_year_before} V GFPower', f'{str_year_before} V GAPower', f'{str_year_before} VMPower']

data = data.merge(vis_power_rankings_df, on='Visitor', how='left')
data = data.merge(home_power_rankings_df, on='Home', how='left')
print('Done')
data

Removing year columns we do not need
Adding Times and Dates
Adding Who Won Last
Adding Win Streaks
Adding which team won last
Adding power rankings.
Done


,Date,Start (ET),Visitor,Vis PTS,Home,Home PTS,2016-17 Vis Rank,2016-17 Home Rank,2017-18 Vis Rank,2017-18 Home Rank,...,2017 V WPower,2017 V LPower,2017 V GFPower,2017 V GAPower,2017 VMPower,2017 H WPower,2017 H LPower,2017 H GFPower,2017 H GAPower,2017 H MPower
0,2018-03-15,9:00p,Phoenix Suns,88,Utah Jazz,116,29,8,NaN,NaN,...,0.032258,0.967742,0.322581,0.967742,0.032258,0.693548,0.306452,0.387097,0.048387,0.838710
1,2019-02-06,9:00p,Phoenix Suns,88,Utah Jazz,116,29,8,30.0,11.0,...,0.032258,0.967742,0.322581,0.967742,0.032258,0.693548,0.306452,0.387097,0.048387,0.838710
2,2018-10-16,8:00p,Philadelphia 76ers,87,Boston Celtics,105,27,4,5.0,4.0,...,0.838710,0.161290,0.774194,0.354839,0.870968,0.870968,0.129032,0.354839,0.096774,0.806452
3,2018-10-16,10:30p,Oklahoma City Thunder,100,Golden State Warriors,108,10,1,10.0,3.0,...,0.693548,0.306452,0.612903,0.322581,0.774194,0.903226,0.096774,0.967742,0.580645,0.903226
4,2018-10-17,7:00p,Milwaukee Bucks,113,Charlotte Hornets,112,14,20,16.0,20.0,...,0.500000,0.500000,0.516129,0.516129,0.354839,0.354839,0.645161,0.677419,0.629032,0.451613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,2019-05-30,9:00p,Golden State Warriors,109,Toronto Raptors,118,1,7,3.0,2.0,...,0.903226,0.096774,0.967742,0.580645,0.903226,0.935484,0.064516,0.887097,0.225806,0.935484
1310,2019-06-02,8:00p,Golden State Warriors,109,Toronto Raptors,104,1,7,3.0,2.0,...,0.903226,0.096774,0.967742,0.580645,0.903226,0.935484,0.064516,0.887097,0.225806,0.935484
1311,2019-06-05,9:00p,Toronto Raptors,123,Golden State Warriors,109,7,1,2.0,3.0,...,0.935484,0.064516,0.887097,0.225806,0.935484,0.903226,0.096774,0.967742,0.580645,0.903226
1312,2019-06-07,9:00p,Toronto Raptors,105,Golden State Warriors,92,7,1,2.0,3.0,...,0.935484,0.064516,0.887097,0.225806,0.935484,0.903226,0.096774,0.967742,0.580645,0.903226


In [23]:
# Create X and y

y = data['Home Win']
X = data.loc[:,'Home PTS':]
X.drop(['Home Odds', 'Vis Odds', 'Home Win', 'Home PTS'], axis=1, inplace=True)
X = X.astype(float)
X.columns

Index(['2016-17 Vis Rank', '2016-17 Home Rank', '2017-18 Vis Rank',
       '2017-18 Home Rank', 'Year', 'Month', 'Day', 'Start (ET)',
       'HomeLastWin', 'VisitorLastWin', 'HomeWinStreak', 'VisitorWinStreak',
       'HomeTeamWonLast', '2017 V WPower', '2017 V LPower', '2017 V GFPower',
       '2017 V GAPower', '2017 VMPower', '2017 H WPower', '2017 H LPower',
       '2017 H GFPower', '2017 H GAPower', '2017 H MPower'],
      dtype='object')

In [24]:
# Train model function

def train_model(X_train_and_test, y_train_and_test, model, test_size):
    ''' Scale, Split, Impute and Train one model '''
    
    X_train, X_test, y_train, y_test = train_test_split(X_train_and_test, y_train_and_test, test_size=test_size, shuffle=False)


    # Impute
    colsT = X_train.columns
    colsV = X_test.columns
    X_train = pd.DataFrame(imputer.fit_transform(X_train))
    X_test = pd.DataFrame(imputer.transform(X_test))
    X_train.columns = colsT
    X_test.columns = colsV
    
    # Scale and replace column names
    X_scaled_train = MMScaler.fit_transform(X_train)
    X_scaled_test = MMScaler.transform(X_test)
    X_scaled_train = pd.DataFrame(X_scaled_train, columns=colsT)
    X_scaled_test = pd.DataFrame(X_scaled_test, columns=colsV)
    
    # Train
    model.fit(X_scaled_train, y_train)
    preds = model.predict(X_scaled_test)

    
    # Combine predictions with actuals
    preds_df = pd.DataFrame(preds, columns=['Predictions'])
    preds_df.index = pd.RangeIndex(start=y_train.last_valid_index()+1, stop=y_train.last_valid_index()+1 + len(y_test))
    predictions_array.append(preds_df)
    preds_and_true = pd.concat([y_test, preds_df], axis=1, ignore_index=True)

    
    # Accuracy
    wins = preds_and_true.apply(lambda x: True if x[0] == True and x[1] == True else False, axis=1)
    losses = preds_and_true.apply(lambda x: True if x[0] == False and x[1] == False else False, axis=1)
    print('Model: ',str(model))
    print('Total test games: ', len(y_test))
    print('Wins predicted correctly: ',len(wins[wins == True].index))
    print('Losses predicted correctly: ',len(losses[losses == True].index))
    print('Percentage predicted correctly: ', (len(wins[wins == True].index) + len(losses[losses == True].index)) / len(preds_and_true))
    
    return preds_df

In [25]:
X.columns
X_power = X.loc[:,f'{str_year_before} V WPower':]
X_power.columns

Index(['2017 V WPower', '2017 V LPower', '2017 V GFPower', '2017 V GAPower',
       '2017 VMPower', '2017 H WPower', '2017 H LPower', '2017 H GFPower',
       '2017 H GAPower', '2017 H MPower'],
      dtype='object')

In [26]:
X.columns
X_no_power = X.loc[:,:f'{str_year_before} V WPower']

In [38]:
# Train and test models

################################################# DF NOT FINDING YEAR WHEN STRING OR INT. TRY MAKE YEAR INT FROM THE START TO AUTO PICK TRAIN AND TEST

str_year = str(year)[:4]

if not use_last_year_data:
    start_month = months_to_train + int(start_month)
    print(start_month)
    if start_month > 12:
        start_month = start_month - 12
        str_year = int(year[:4]) + 1
        print(start_month)

# Find train cut off
train_test_cut_off = X[(X['Year'] == int(str_year)) & (X['Month'] == int(start_month))].index[20] / len(X) # Increase index if error: Length mismatch: Expected axis has 11 elements, new values have 13 elements

predictions_array = []

models_array = [svc, ADBC, RFC, GBC, HGBC, XGB, QDA, KNC]

for model in models_array:
    train_model(X_no_power, y, model, 1 - train_test_cut_off)

Model:  SVC(C=10, gamma=0.1)
Total test games:  1292
Wins predicted correctly:  481
Losses predicted correctly:  206
Percentage predicted correctly:  0.531733746130031
Model:  AdaBoostClassifier(learning_rate=0.8, n_estimators=155)
Total test games:  1292
Wins predicted correctly:  448
Losses predicted correctly:  235
Percentage predicted correctly:  0.5286377708978328
Model:  RandomForestClassifier(min_samples_split=5, n_estimators=1000)
Total test games:  1292
Wins predicted correctly:  632
Losses predicted correctly:  123
Percentage predicted correctly:  0.5843653250773994
Model:  GradientBoostingClassifier(learning_rate=0.15, n_estimators=500)
Total test games:  1292
Wins predicted correctly:  448
Losses predicted correctly:  258
Percentage predicted correctly:  0.5464396284829721
Model:  HistGradientBoostingClassifier(l2_regularization=1.5, max_leaf_nodes=80,
                               min_samples_leaf=25)
Total test games:  1292
Wins predicted correctly:  762
Losses predicted

In [39]:
# CREATE BACKTEST DATAFRAME TO WORK WITH

# Take necessary columns from data df
backtest_data = data[['Date', 'Visitor', 'Home', 'Home Odds', 'Vis Odds', 'Home Win']]

# Make the predictions a DF
for i in predictions_array:
    df = pd.DataFrame(i)
    backtest_data = pd.concat([backtest_data, df], axis=1)

# Rename the columns
backtest_data.columns = ['Date', 'Visitor', 'Home', 'Home Odds', 'Vis Odds', 'Home Win', 'SVC', 'ADBC', 'RFC', 'GBC', 'HGBC', 'XGB', 'QDA', 'KNC']

# Only keep the rows with predictions
rows_with_odds = len(backtest_data[backtest_data['SVC'].isnull()]) + 1
backtest_data = backtest_data.iloc[rows_with_odds:,:]

In [40]:
## Create all column for limit bankroll = cumsum axis=1 for limit rows
backtest_data.loc[:,'SVC':'KNC']
backtest_data['True Count'] = backtest_data.loc[:,'SVC':'KNC'].any(axis=1)

In [41]:
backtest_data.loc[:,'SVC':'KNC']

,SVC,ADBC,RFC,GBC,HGBC,XGB,QDA,KNC
23,False,False,True,True,True,True,False,True
24,False,False,False,True,True,False,False,False
25,True,False,False,False,True,False,False,True
26,False,False,True,True,True,True,False,False
27,True,True,True,True,True,True,False,True
...,...,...,...,...,...,...,...,...
1309,True,False,True,True,True,True,False,False
1310,True,True,True,True,True,True,False,True
1311,True,True,True,True,True,True,False,True
1312,True,True,True,True,True,True,False,True


In [42]:
(backtest_data.loc[:,'SVC':'KNC'] == False).any(axis=1).value_counts()

True     1275
False      16
dtype: int64

In [43]:
backtest_data['True Count'].value_counts()

True    1291
Name: True Count, dtype: int64

In [44]:
# CALCULATE BANKROLL

# Define function for outcome
def bankroll_calc(col):
    
    conditions = [
        (backtest_data[col] != backtest_data['Home Win']),
        ((backtest_data[col] == False) & (backtest_data['Home Win'] == False)),
        ((backtest_data[col] == True) & (backtest_data['Home Win'] == True))
    ]

    values = [-1, backtest_data['Vis Odds'] - 1, backtest_data['Home Odds'] -1]

    backtest_data[col + ' Outcome'] = np.select(conditions, values)

col_array = ['SVC', 'ADBC', 'RFC', 'GBC', 'HGBC', 'XGB', 'QDA', 'KNC']

# Generate outcomes
for col in col_array:
    bankroll_calc(col)

# calculate bankroll
for col in col_array:
    backtest_data[col + ' Bankroll'] = backtest_data[col + ' Outcome'].cumsum(axis=0)
    
no_limit_df = backtest_data.loc[:,'SVC Bankroll':'KNC Bankroll']
no_limit_df['Date'] = backtest_data['Date']

backtest_data

,Date,Visitor,Home,Home Odds,Vis Odds,Home Win,SVC,ADBC,RFC,GBC,...,QDA Outcome,KNC Outcome,SVC Bankroll,ADBC Bankroll,RFC Bankroll,GBC Bankroll,HGBC Bankroll,XGB Bankroll,QDA Bankroll,KNC Bankroll
23,2018-10-19,Boston Celtics,Toronto Raptors,1.67,2.29,True,False,False,True,True,...,-1.00,0.67,-1.00,-1.00,0.67,0.67,0.67,0.67,-1.00,0.67
24,2018-10-19,Indiana Pacers,Milwaukee Bucks,1.57,2.47,True,False,False,False,True,...,-1.00,-1.00,-2.00,-2.00,-0.33,1.24,1.24,-0.33,-2.00,-0.33
25,2018-10-19,Oklahoma City Thunder,Los Angeles Clippers,1.66,2.28,True,True,False,False,False,...,-1.00,0.66,-1.34,-3.00,-1.33,0.24,1.90,-1.33,-3.00,0.33
26,2018-10-19,Golden State Warriors,Utah Jazz,2.15,1.73,False,False,False,True,True,...,0.73,0.73,-0.61,-2.27,-2.33,-0.76,0.90,-2.33,-2.27,1.06
27,2018-10-20,Brooklyn Nets,Indiana Pacers,1.26,4.05,True,True,True,True,True,...,-1.00,0.26,-0.35,-2.01,-2.07,-0.50,1.16,-2.07,-3.27,1.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,2019-05-30,Golden State Warriors,Toronto Raptors,1.78,2.09,True,True,False,True,True,...,-1.00,-1.00,-76.98,-68.65,-62.69,-86.10,-26.36,-41.09,-49.75,-75.01
1310,2019-06-02,Golden State Warriors,Toronto Raptors,1.75,2.13,False,True,True,True,True,...,1.13,-1.00,-77.98,-69.65,-63.69,-87.10,-27.36,-42.09,-48.62,-76.01
1311,2019-06-05,Toronto Raptors,Golden State Warriors,1.66,2.27,False,True,True,True,True,...,1.27,-1.00,-78.98,-70.65,-64.69,-88.10,-28.36,-43.09,-47.35,-77.01
1312,2019-06-07,Toronto Raptors,Golden State Warriors,1.47,2.78,False,True,True,True,True,...,1.78,-1.00,-79.98,-71.65,-65.69,-89.10,-29.36,-44.09,-45.57,-78.01


In [45]:
def plot(df, col_name):
    
    # Visualize the data
    p = figure(x_axis_type='datetime', plot_width=1200, 
               plot_height=800, title='Bankroll',
              x_axis_label='Time', y_axis_label='Money ($)')

    source = ColumnDataSource(df)
    source.data.keys()

    color_dict ={
        'SVC':'lime',
        'ADBC':'red',
        'RFC':'blue',
        'GBC':'purple',
        'HGBC':'orange',
        'XGB':'yellow',
        'QDA':'magenta',
        'KNC':'black'
    }

    for col in col_array:    
        col_line = p.line(y=col + ' ' + col_name, x='Date', source=source, legend_label=col, line_color=color_dict.get(col))
        col_line
        p.add_tools(HoverTool(renderers=[col_line], mode='vline', formatters={'@Date': 'datetime'}, tooltips=[('Date','@Date{%F}'), (col, '$@{' + col + ' ' + col_name+'}')]))

    p.legend.click_policy='hide'

    show(p)

In [46]:
# CALCULATE BANKROLL WITH ODDS LIMIT

odds_limit = 3
odds_high_limit = 7
# Define function for outcome

col = 'SVC'
def bankroll_calc_limit(col):    
    conditions = [
        ((backtest_data['Vis Odds'] > odds_limit) & (backtest_data['Vis Odds'] < odds_high_limit) & (backtest_data[col] == False) & (backtest_data['Home Win'] == True)),
        ((backtest_data['Home Odds'] > odds_limit) & (backtest_data['Home Odds'] < odds_high_limit) & (backtest_data[col] == True) & (backtest_data['Home Win'] == False)),
        ((backtest_data['Vis Odds'] > odds_limit) & (backtest_data['Vis Odds'] < odds_high_limit) & (backtest_data[col] == False) & (backtest_data['Home Win'] == False)),
        ((backtest_data['Home Odds'] > odds_limit) & (backtest_data['Home Odds'] < odds_high_limit) & (backtest_data[col] == True) & (backtest_data['Home Win'] == True)),
        (True)
    ]

    values = [-1, -1, backtest_data['Vis Odds'] - 1, backtest_data['Home Odds'] - 1, 0]

    backtest_data[col + ' Limit Outcome'] = np.select(conditions, values)

col_array = ['SVC', 'ADBC', 'RFC', 'GBC', 'HGBC', 'XGB', 'QDA', 'KNC']

# Generate outcomes
for col in col_array:
    bankroll_calc_limit(col)

# calculate bankroll
for col in col_array:
    backtest_data[col + ' Limit Bankroll'] = backtest_data[col + ' Limit Outcome'].cumsum(axis=0)
    
limit_df = backtest_data.loc[:,'SVC Limit Bankroll':'KNC Limit Bankroll']
limit_df['Date'] = backtest_data['Date']

limit_df.describe()

backtest_data.to_excel('backtest_data.xlsx')

In [47]:
no_limit_df.describe()

,SVC Bankroll,ADBC Bankroll,RFC Bankroll,GBC Bankroll,HGBC Bankroll,XGB Bankroll,QDA Bankroll,KNC Bankroll
count,1291.000000,1291.000000,1291.000000,1291.000000,1291.000000,1291.000000,1291.000000,1291.000000
mean,-40.998799,-55.577180,-29.713888,-47.742649,-5.960488,-29.201642,-33.920426,-41.092014
std,21.818584,19.713781,15.077803,20.433567,11.453559,10.909678,21.769771,20.119102
min,-82.720000,-82.780000,-66.690000,-90.100000,-30.650000,-49.850000,-79.280000,-80.190000
25%,-57.775000,-70.935000,-40.445000,-60.485000,-13.110000,-36.305000,-49.025000,-59.405000
50%,-39.290000,-62.330000,-32.380000,-54.360000,-6.050000,-30.430000,-39.610000,-35.580000
75%,-28.960000,-44.160000,-16.660000,-37.485000,3.300000,-25.280000,-16.830000,-24.710000
max,0.720000,0.030000,1.470000,1.240000,14.710000,0.670000,10.600000,1.320000


In [48]:
plot(limit_df, 'Limit Bankroll')

In [21]:
plot(no_limit_df, 'Bankroll')